# Macro Signal Strategy Analysis

This notebook provides interactive analysis of macro trading strategies.

In [ ]:
# Setup
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

from src.data.macro_data_loader import MacroDataLoader
from src.data.asset_price_loader import AssetPriceLoader
from src.signals.yield_curve_signal import YieldCurveSignal
from src.backtest.performance import PerformanceAnalyzer

# Set style
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:
# Load configuration
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Initialize data loaders
macro_loader = MacroDataLoader(fred_api_key=config['data']['fred_api_key'])
asset_loader = AssetPriceLoader()

## 1. Macro Data Analysis

In [ ]:
# Fetch macro data
start_date = "2010-01-01"
end_date = "2023-12-31"

macro_data = macro_loader.fetch_multiple_series(
    ['yield_10y', 'yield_2y', 'cpi', 'gdp'], 
    start_date, 
    end_date
)

In [ ]:
# Plot yield curve
fig, ax = plt.subplots(figsize=(12, 6))

spread = macro_data['yield_10y'] - macro_data['yield_2y']
spread.plot(ax=ax, label='10Y-2Y Spread')

ax.axhline(y=0, color='red', linestyle='--', alpha=0.7)
ax.set_title('Yield Curve Slope Over Time')
ax.set_ylabel('Spread (%)')
ax.legend()
plt.tight_layout()
plt.show()

## 2. Signal Analysis

In [ ]:
# Generate yield curve signal
yield_signal = YieldCurveSignal()
signal_values = yield_signal.generate_signal(macro_data)

# Plot signal
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Raw spread
spread.plot(ax=ax1, label='Raw Spread')
ax1.set_ylabel('Spread (%)')
ax1.set_title('Yield Curve Spread')
ax1.legend()

# Normalized signal
signal_values.plot(ax=ax2, label='Normalized Signal', color='green')
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)
ax2.axhline(y=0.5, color='orange', linestyle=':', alpha=0.5)
ax2.axhline(y=-0.5, color='orange', linestyle=':', alpha=0.5)
ax2.set_ylabel('Signal Value')
ax2.set_title('Normalized Trading Signal')
ax2.legend()

plt.tight_layout()
plt.show()

## 3. Asset Performance Analysis

In [ ]:
# Fetch asset prices
assets = ['SPY', 'TLT', 'GLD']
price_data = asset_loader.fetch_multiple_assets(assets, start_date, end_date)

# Calculate returns
returns = asset_loader.calculate_returns(price_data)

# Plot cumulative returns
cumulative_returns = (1 + returns).cumprod()

fig, ax = plt.subplots(figsize=(12, 6))
cumulative_returns.plot(ax=ax)
ax.set_title('Cumulative Returns by Asset')
ax.set_ylabel('Cumulative Return')
ax.legend()
plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Create correlation matrix
# Align signal data with returns
aligned_signal = signal_values.reindex(returns.index, method='ffill')

# Combine for correlation
combined_data = pd.concat([
    returns,
    aligned_signal.to_frame('yield_signal')
], axis=1)

# Calculate correlation
correlation_matrix = combined_data.corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
ax.set_title('Correlation Matrix: Assets and Signal')
plt.tight_layout()
plt.show()

## 5. Rolling Performance Analysis

In [ ]:
# Calculate rolling Sharpe ratios
window = 252  # 1 year

rolling_sharpe = returns.rolling(window).apply(
    lambda x: (x.mean() / x.std()) * np.sqrt(252) if x.std() != 0 else 0
)

fig, ax = plt.subplots(figsize=(12, 6))
rolling_sharpe.plot(ax=ax)
ax.set_title(f'{window}-Day Rolling Sharpe Ratio')
ax.set_ylabel('Sharpe Ratio')
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()